In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
sentiment_df = pd.read_csv("fear_greed_index.csv")
trades_df = pd.read_csv("Historical_Data.csv")

# Preview data
print(sentiment_df.head())
print(trades_df.head())


In [ ]:
# Convert date columns to datetime
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
trades_df['Timestamp IST'] = pd.to_datetime(trades_df['Timestamp IST'])

# Extract just the date for merging
sentiment_df['merge_date'] = sentiment_df['date'].dt.date
trades_df['merge_date'] = trades_df['Timestamp IST'].dt.date


In [ ]:
# Merge datasets on date
merged_df = pd.merge(trades_df, sentiment_df, on='merge_date', how='left')

# Check the result
print(merged_df[['Account', 'Coin', 'Closed PnL', 'classification', 'value']].head())


In [ ]:
# Average profit/loss on Fear vs Greed days
pnl_by_sentiment = merged_df.groupby('classification')['Closed PnL'].mean()
print(pnl_by_sentiment)

# Visualize
sns.boxplot(data=merged_df, x='classification', y='Closed PnL')
plt.title('Profit/Loss Distribution by Market Sentiment')
plt.ylabel('Closed PnL')
plt.xlabel('Sentiment')
plt.grid(True)
plt.show()


In [ ]:
# Average PnL by sentiment and trade direction
pnl_by_dir = merged_df.groupby(['classification', 'Direction'])['Closed PnL'].mean().unstack()
print(pnl_by_dir)

# Visualize
pnl_by_dir.plot(kind='bar')
plt.title('Avg PnL by Sentiment and Direction')
plt.ylabel('Average PnL')
plt.grid(True)
plt.show()


In [ ]:
# Scatter plot and correlation
sns.scatterplot(data=merged_df, x='value', y='Closed PnL', hue='classification')
plt.title("Sentiment Score vs PnL")
plt.grid(True)
plt.show()

# Correlation value
correlation = merged_df['value'].corr(merged_df['Closed PnL'])
print(f"Correlation between sentiment value and PnL: {correlation:.2f}")


In [ ]:
if 'leverage' in merged_df.columns:
    avg_leverage = merged_df.groupby('classification')['leverage'].mean()
    print(avg_leverage)
    avg_leverage.plot(kind='bar', title='Avg Leverage by Sentiment')
    plt.ylabel('Leverage')
    plt.grid(True)
    plt.show()


In [ ]:
# Top performing accounts during Greed and Fear
top_accounts = merged_df.groupby(['Account', 'classification'])['Closed PnL'].sum().unstack().fillna(0)
print(top_accounts.sort_values('Greed', ascending=False).head())
print(top_accounts.sort_values('Fear', ascending=False).head())


In [ ]:
# Save the merged dataset
merged_df.to_csv("merged_trader_sentiment.csv", index=False)
